# Project Universe: A Complete Technical Guide

This document provides a comprehensive explanation of your interactive 3D portfolio. It covers the project's architecture, the libraries used, the core logic, and a step-by-step guide for future modifications.

---

## 1. High-Level Architecture

At its core, this project is a **single-page application (SPA)** that uses 3D graphics to create an immersive and interactive representation of your resume. The architecture is designed to be modular and scalable.

### File Structure
The project is cleanly separated into three main files, each with a distinct responsibility:

* **`index.html`**: The skeleton of the application. It defines the structure of the webpage, including the canvas for the 3D scene, the hero section, the sidebar for information, and the spaceship animation overlay. It's also responsible for loading all external libraries and linking our CSS and JavaScript files.
* **`style.css`**: The visual skin of the application. This file contains all the custom styling rules that are not handled by Tailwind CSS. This includes the futuristic font styles, the sidebar's blurred glass effect, and the keyframe animations for the spaceship and sparks.
* **`script.js`**: The brain of the application. This is where all the dynamic and interactive logic resides. It handles everything from setting up the 3D world and animating the celestial bodies to managing user clicks and camera movements.

---

## 2. Libraries & Technologies Explained

This project leverages several powerful open-source libraries to achieve its functionality.

### Three.js (The 3D Engine)
**Three.js** is the heart of the visual experience. It's a JavaScript library that simplifies the process of creating and displaying 3D computer graphics in a web browser. It uses **WebGL** under the hood but provides a much more developer-friendly API.

Here are the key components of Three.js used in our project:

* **`Scene`**: Think of this as a virtual stage or universe where all your 3D objects, lights, and cameras live.
* **`Camera` (`PerspectiveCamera`)**: This is the viewer's eye in the 3D world. A `PerspectiveCamera` mimics how the human eye sees, making objects appear smaller the farther away they are.
* **`Renderer` (`WebGLRenderer`)**: This is the engine that takes the `Scene` and `Camera` as input and draws (renders) the resulting 2D image onto the HTML `<canvas>` element for the user to see.
* **`Mesh`**: A `Mesh` is the fundamental object for creating visible things in the scene. It's a combination of a `Geometry` and a `Material`.
    * **`Geometry` (`SphereGeometry`, `CylinderGeometry`, etc.)**: This defines the shape of an object (e.g., a sphere for a planet, a cylinder for a flag pole).
    * **`Material` (`MeshStandardMaterial`, `MeshBasicMaterial`)**: This defines the appearance of an object's surface (e.g., its color, texture, shininess, or whether it's affected by light). `MeshStandardMaterial` is used for realistic objects that react to light, while `MeshBasicMaterial` is used for simple, unlit objects like the sun.
* **`Light` (`PointLight`, `AmbientLight`)**: Lights are essential for illuminating the scene.
    * `PointLight`: Emits light in all directions from a single point, like a light bulb. This creates shadows and gives depth to the scene.
    * `AmbientLight`: Provides a soft, base level of light to the entire scene, ensuring that even parts in shadow are not completely black.
* **`Raycaster`**: This is a powerful tool used for "picking" or detecting mouse interactions with 3D objects. It works by casting an invisible ray from the camera through the mouse's position on the screen and reporting which objects the ray intersects. This is how we know when you click on a planet.

### Tailwind CSS (The Styling Framework)
**Tailwind CSS** is a utility-first CSS framework. Instead of writing custom CSS classes like `.big-blue-button`, you apply pre-existing, single-purpose utility classes directly in your HTML (e.g., `bg-blue-500`, `text-white`, `p-4`). This makes styling rapid and consistent. It's used for almost all the layout, sizing, and coloring of the HTML elements.

### GSAP (The Animation Platform)
**GSAP (GreenSock Animation Platform)** is the industry standard for high-performance JavaScript animation. While simple animations can be done with CSS, GSAP provides precise control over complex sequences, which is perfect for our camera movements.

* **Why GSAP?**: It allows us to smoothly animate the camera's `position` and its "look-at" target simultaneously over a set duration with custom easing (`"power3.inOut"`), creating a fluid, cinematic transition that would be very difficult to achieve otherwise.

---

## 3. Core Logic Explained (`script.js`)

The `script.js` file can be broken down into several logical sections.

### a. Data-Driven Content (`universeData`)
The entire portfolio's content is stored in a single JavaScript object called `universeData`. This is a crucial design choice.

* **Why?**: It completely separates the **data** (your resume information) from the **presentation** (the 3D logic). This means you can update your projects, skills, or experiences without ever touching the complex animation or rendering code.
* **Structure**: The object has a `sun` key for your "About Me" section and a `planets` array. Each planet object has a `name` and either a `content` property (for planets that directly show information) or a `moons` array for planets with sub-categories.

### b. State Management
To keep track of what's happening in the application, we use a few simple "state" variables:

* `activeView`: A string that tells us if we are in the main `'solarSystem'` view or the focused `'moonSystem'` view. The `animate()` loop behaves differently based on this state.
* `focusedPlanet`: Stores the Three.js object of the planet the user has clicked on to view its moons.
* `isAnimating`: A boolean flag that is set to `true` during camera transitions. This is used to prevent the user from clicking on objects while the camera is moving, which avoids bugs and creates a smoother experience.

### c. Interaction and Animation Flow
This is the sequence of events when a user interacts with the scene:

1.  **Mouse Move (`onMouseMove`)**: The `Raycaster` is constantly updated with the mouse's position. If it detects an intersection with a celestial body, the cursor changes to a pointer.
2.  **Mouse Click (`onMouseClick`)**:
    * The `Raycaster` fires to see what was clicked.
    * The code checks the `userData` of the clicked object.
    * **If it's a planet with moons**: The `focusOnPlanet()` function is called.
    * **If it's a sun, a moon, or a planet without moons**: The `triggerSpaceshipAnimation()` function is called.
3.  **`triggerSpaceshipAnimation(data)`**:
    * The animation overlay is made visible.
    * The spaceship's CSS animation (`fly-in-out`) is started.
    * The spark effect is generated dynamically using JavaScript.
    * A `setTimeout` is used to call `openSidebar(data)` after a short delay, so the information appears while the spaceship is still on screen.
    * Another `setTimeout` cleans up and hides the overlay after the animation is complete.
4.  **`focusOnPlanet(planet)`**:
    * The `isAnimating` flag is set to `true`.
    * GSAP is used to smoothly animate the camera's position to a point near the clicked planet. The `onUpdate` callback ensures the camera keeps looking at the planet throughout the transition.
    * While the camera is moving, other planets and the sun are faded out using GSAP to animate their material's opacity.
    * In the `onComplete` callback (once the camera arrives), the `activeView` is switched to `'moonSystem'`, the "Back" button appears, and the `createMoonsForPlanet()` function is called.
5.  **`returnToSolarSystem()`**: This function does the reverse of `focusOnPlanet()`, using GSAP to fly the camera back to its original position and fading the planets back in.

---

## 4. How to Add New Content (Step-by-Step)

Adding new content is designed to be simple. You only need to modify the `universeData` and `planetInfo` objects in `script.js`.

### How to Add a New Planet (e.g., "Hobbies")

Let's say you want to add a new planet called "Hobbies" that directly shows information in the sidebar.

1.  **Open `script.js`**.
2.  **Update `universeData`**: Go to the `planets` array inside the `universeData` object. Add a new object for your planet.

    ```javascript
    // ... inside universeData.planets array
    {
        name: "Achievements",
        content: `<ul class="list-disc list-inside space-y-3">...</ul>`
    },
    // START: Add your new planet here
    {
        name: "Hobbies",
        content: `<p>In my free time, I enjoy exploring photography, hiking, and competitive chess.</p>`
    },
    // END: New planet added
    { 
        name: "Certificates",
        moons: [ /* ... */ ]
    }
    ```

3.  **Update `planetInfo`**: Go to the `planetInfo` array. This array controls the visual appearance and orbit of each planet, and its order **must match** the order in `universeData.planets`. Add a new object for your planet's visuals.

    ```javascript
    // ... inside planetInfo array
    { color: 0xF1C40F, size: 0.45, orbitRadius: 14.0 }, // Achievements
    // START: Add visuals for your new planet
    { color: 0x2ECC71, size: 0.4, orbitRadius: 15.0 },  // Hobbies (Green, small, new orbit)
    // END: New planet visuals added
    { color: 0x00FFD0, size: 0.5, orbitRadius: 16.5 }  // Certificates (Adjusted orbit)
    ```
    **Important**: Notice that I had to increase the `orbitRadius` of the "Certificates" planet to make room for the new "Hobbies" planet. You must ensure the orbit radii are always increasing to prevent planets from overlapping.

That's it! The program will automatically create the new planet with its flag and make it interactive.

### How to Add a New Moon to an Existing Planet

Let's say you want to add a new project, "Portfolio Website," to the "Projects" planet.

1.  **Open `script.js`**.
2.  **Find the Planet**: Locate the "Projects" object inside the `universeData.planets` array.
3.  **Add to the `moons` array**: Add a new object to its `moons` array.

    ```javascript
    // ... inside the "Projects" planet object
    moons: [
        { name: "Reelify", content: `...` },
        { name: "Text2Sign", content: `...` },
        { name: "Football Analysis", content: `...` },
        // START: Add your new moon here
        { 
            name: "Portfolio Website", 
            content: `<p>An interactive 3D portfolio built with Three.js and GSAP to showcase my skills and projects in a unique, engaging way.</p><div class="flex flex-wrap gap-2 mb-4"><span class="tag">Three.js</span><span class="tag">GSAP</span><span class="tag">TailwindCSS</span></div>`
        }
        // END: New moon added
    ]
    ```

The program will automatically create this new moon orbiting the "Projects" planet when you click on it. The orbit radius and spacing are calculated automatically.

This data-centric approach ensures that your portfolio can grow with your career. Congratulations on building such a dynamic and impressive project!
